In [1]:
%pip install mlflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import digitalhub as dh

project = dh.get_or_create_project("demo-mlflow-ml_model")

In [2]:
import os
directory="src"
if not os.path.exists(directory):
    os.makedirs(directory)

In [3]:
%%writefile "src/train-model.py"

from digitalhub_runtime_python import handler
from digitalhub import from_mlflow_run
import mlflow

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

@handler()
def train(project):
    mlflow.sklearn.autolog(log_datasets=True)

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # utility to map mlflow run artifacts to model metadata
    model_params = from_mlflow_run(run_id)

    project.log_model(
        name="model-mlflow",
        kind="mlflow",
        **model_params
)

Writing src/train-model.py


In [4]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train",
                                requirements=["scikit-learn", "mlflow"])

In [5]:
train_run = train_fn.run(action="job", local_execution=False)

In [6]:
model = project.get_model("model-mlflow")
model.spec.path

's3://datalake/demo-mlflow-ml_model/model/model-mlflow/fed2dd32-6a76-4b99-90a7-dcced849ac2a/'

/v2/models/model/infer

In [7]:
func = project.new_function(name="serve_mlflowmodel",
                            kind="mlflowserve",
                            model_name="mlflow_model",
                            path=model.spec.path + 'model/')

In [8]:
serve_run = func.run(action="serve")

Let us wait till the 'serve' is loaded compeletey

In [9]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

In [11]:
json

{'inputs': [{'name': 'input-0',
   'shape': [-1, 4],
   'datatype': 'FP64',
   'data': [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2]]}]}

In [3]:
# serve_run.invoke(model_name="mlflow_model", json=json).json()

In [11]:
service_url = serve_run.refresh().status.to_dict()["service"]["url"]
url = f"http://{service_url}/v2/models/mlflow_model/infer"
url

'http://s-mlflowserveserve-d8e99825-cb84-460f-ba04-8582ad72c024.digitalhub-tenant1:8080/v2/models/mlflow_model/infer'

<!-- /v2/models/model/infer -->

In [12]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

json

{'inputs': [{'name': 'input-0',
   'shape': [-1, 4],
   'datatype': 'FP64',
   'data': [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2]]}]}

In [13]:
# url = 's-mlflowserveserve-66f056e7-99b9-481e-858d-d9888df35f91.digitalhub-tenant2:8080'

In [14]:
serve_run.invoke(url=url, json=json).json()
# serve_run.invoke(model_name="mlflow_model", url=url, json=json).json()

{'model_name': 'mlflow_model',
 'id': '5562db0b-e29b-4d27-839e-6898acf27e42',
 'parameters': {'content_type': 'np'},
 'outputs': [{'name': 'output-1',
   'shape': [2, 1],
   'datatype': 'INT64',
   'parameters': {'content_type': 'np'},
   'data': [0, 0]}]}